In [1]:
import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np
import time
import pprint
from tqdm.notebook import trange

from sac import Agent
from obs import observation_shape
from utils import record_videos, load_config

filename = 'inverted_pendulum.png'

best_score = -10000.0
best_score = -1000.0
score_history = []
episode_lens = []
avg_history = []
std_history = []
avg_history_100 = []

2023-12-13 19:29:11.373686: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-13 19:29:11.614751: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-13 19:29:11.614814: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-13 19:29:11.616119: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-13 19:29:11.728278: I tensorflow/core/platform/cpu_feature_g

In [2]:
data = load_config()
env = gym.make('racetrack-v0', render_mode = 'rgb_array')
env.configure(data) # type: ignore
pprint.pprint(env.config) # type: ignore
(obs, info), done = env.reset(), False

#observation config
proc = observation_shape(obs,info,2)
proc.reset()
input = proc.get_input()
print(input.shape)


{'action': {'lateral': True, 'longitudinal': True, 'type': 'ContinuousAction'},
 'action_reward': -0.3,
 'centering_position': [0.7, 0.7],
 'collision_reward': -1,
 'controlled_vehicles': 1,
 'duration': 80,
 'lane_centering_cost': 4,
 'lane_centering_reward': 1,
 'manual_control': False,
 'observation': {'features': ['presence',
                              'x',
                              'y',
                              'vx',
                              'vy',
                              'cos_h',
                              'sin_h',
                              'heading',
                              'long_off',
                              'lat_off',
                              'ang_off'],
                 'features_range': {'vx': [-20, 20],
                                    'vy': [-20, 20],
                                    'x': [-100, 100],
                                    'y': [-100, 100]},
                 'type': 'Kinematics',
                 'vehicles_c

/home/o/Documents/thesis/.venv/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.configure to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.configure` for environment variables or `env.get_wrapper_attr('configure')` that will search the reminding wrappers.
  logger.warn(
/home/o/Documents/thesis/.venv/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.config to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.config` for environment variables or `env.get_wrapper_attr('config')` that will search the reminding wrappers.
  logger.warn(


In [4]:
agent = Agent(input_dims=input.shape, env=env,
            n_actions=2)

In [5]:
# main loop
for episode in trange(500, desc='Test episodes'):

        proc.reset()        
        (observation, info), done = env.reset(), False
        truncated = False
        
        proc.update_input(observation, info)
        observation = proc.get_input()

        episode_reward = 0
        episode_len = 0

        while (not done) and (not truncated):
            action = agent.choose_action(observation)

            new_observation, reward, done, truncated, new_info = env.step(action=action)
            proc.update_input(new_observation, info)
            new_observation = proc.get_input()

            episode_reward += reward # type: ignore
            episode_len +=1

            if new_info["rewards"]["on_road_reward"] == False or episode_len >= 200:
                truncated = True

            agent.remember(state=observation, action=action, done=done,
                            reward=reward, new_state=new_observation)
            agent.learn()

            observation = new_observation

        episode_lens.append(episode_len)

        score_history.append(episode_reward)
        avg_score = np.mean(score_history)
        avg_history.append(avg_score)
        std_score = np.std(score_history)
        std_history.append(std_score)

        avg_score_100 = np.mean(score_history[-100:])
        avg_history_100.append(avg_score_100)

        agent.save_models(episode)

        agent.tensorboard.update_stats(episode_rew = episode_reward, 
                                       average_rew =avg_score,
                                       average_100_reward = avg_score_100,
                                       std_rew=std_score, 
                                       episode_len = episode_len)

        print('episode ', episode, 'score %.1f' % episode_reward, 'ep len', episode_len,
              'avg score %.1f' % avg_score, 'avg_score_100 %.1f' %avg_score_100,'std score %.1f' % std_score)



Test episodes:   0%|          | 0/500 [00:00<?, ?it/s]

2023-12-13 19:30:15.018111: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


... saving models ...
episode  0 score -1.0 ep len 6 avg score -1.0 avg_score_100 -1.0 std score 0.0
... saving models ...
episode  1 score -1.1 ep len 6 avg score -1.0 avg_score_100 -1.0 std score 0.1
... saving models ...
episode  2 score -1.3 ep len 6 avg score -1.1 avg_score_100 -1.1 std score 0.1
... saving models ...
episode  3 score -0.5 ep len 6 avg score -1.0 avg_score_100 -1.0 std score 0.3
... saving models ...
episode  4 score -0.5 ep len 5 avg score -0.9 avg_score_100 -0.9 std score 0.3
... saving models ...
episode  5 score -0.8 ep len 5 avg score -0.9 avg_score_100 -0.9 std score 0.3
... saving models ...
episode  6 score -0.7 ep len 5 avg score -0.8 avg_score_100 -0.8 std score 0.3
... saving models ...
episode  7 score -0.3 ep len 5 avg score -0.8 avg_score_100 -0.8 std score 0.3
... saving models ...
episode  8 score -0.7 ep len 6 avg score -0.8 avg_score_100 -0.8 std score 0.3
... saving models ...
episode  9 score -0.8 ep len 6 avg score -0.8 avg_score_100 -0.8 std 

2023-12-13 19:30:17.997592: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55dae7203600 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-13 19:30:17.997610: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2023-12-13 19:30:18.003599: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-13 19:30:19.252395: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2023-12-13 19:30:19.307555: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


... saving models ...
episode  18 score -0.4 ep len 3 avg score -0.8 avg_score_100 -0.8 std score 0.3
... saving models ...
episode  19 score -0.2 ep len 5 avg score -0.7 avg_score_100 -0.7 std score 0.3
... saving models ...
episode  20 score -0.5 ep len 3 avg score -0.7 avg_score_100 -0.7 std score 0.3
... saving models ...
episode  21 score -0.4 ep len 3 avg score -0.7 avg_score_100 -0.7 std score 0.3
... saving models ...
episode  22 score -1.0 ep len 6 avg score -0.7 avg_score_100 -0.7 std score 0.3
... saving models ...
episode  23 score 0.6 ep len 3 avg score -0.7 avg_score_100 -0.7 std score 0.4
... saving models ...
episode  24 score -1.2 ep len 11 avg score -0.7 avg_score_100 -0.7 std score 0.4
... saving models ...
episode  25 score -0.3 ep len 3 avg score -0.7 avg_score_100 -0.7 std score 0.4
... saving models ...
episode  26 score -0.5 ep len 4 avg score -0.7 avg_score_100 -0.7 std score 0.4
... saving models ...
episode  27 score -0.6 ep len 7 avg score -0.7 avg_score_100

KeyboardInterrupt: 

In [6]:
n_steps = sum(episode_lens)
print(n_steps)


51648


In [7]:

# env = record_videos(env)
from gymnasium.wrappers import RecordVideo

data = load_config()
env = gym.make('racetrack-v0', render_mode = 'rgb_array')
env.configure(data) # type: ignore

env = RecordVideo(env, video_folder="videos", episode_trigger=lambda e: True)
env.unwrapped.set_record_video_wrapper(env)


for episode in trange(5, desc='Test episodes'):

        proc.reset()        
        (observation, info), done = env.reset(), False
        truncated = False
        
        proc.update_input(observation, info)
        observation = proc.get_input()

        episode_reward = 0
        episode_len = 0

        while (not done) and (not truncated):
            action = agent.choose_action(observation)

            new_observation, reward, done, truncated, new_info = env.step(action=action)
            proc.update_input(new_observation, info)
            new_observation = proc.get_input()

            episode_reward += reward # type: ignore
            episode_len +=1

            if new_info["rewards"]["on_road_reward"] == False or episode_len >= 300:
                truncated = True

            observation = new_observation


env.close()


/home/o/Documents/thesis/.venv/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.configure to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.configure` for environment variables or `env.get_wrapper_attr('configure')` that will search the reminding wrappers.
  logger.warn(


Test episodes:   0%|          | 0/5 [00:00<?, ?it/s]

Moviepy - Building video /home/o/Documents/thesis/two_acts/SAC/videos/rl-video-episode-0.mp4.
Moviepy - Writing video /home/o/Documents/thesis/two_acts/SAC/videos/rl-video-episode-0.mp4



Moviepy - Done !
Moviepy - video ready /home/o/Documents/thesis/two_acts/SAC/videos/rl-video-episode-0.mp4
